# MULTI-LAYER PERCEPTRON WITH CUSTOM DATA

In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline  
print ("Packages loaded")

Packages loaded


# LOAD DATA

In [2]:
# Load them!
cwd = os.getcwd()
loadpath = cwd + "/data/custom_data.npz"
l = np.load(loadpath)

# See what's in here
print (l.files)

# Parse data
trainimg = l['trainimg']
trainlabel = l['trainlabel']
testimg = l['testimg']
testlabel = l['testlabel']
imgsize = l['imgsize']
ntrain = trainimg.shape[0]
nclass = trainlabel.shape[1]
dim    = trainimg.shape[1]
ntest  = testimg.shape[0]
print ("%d train images loaded" % (ntrain))
print ("%d test images loaded" % (ntest))
print ("%d dimensional input" % (dim))
print ("Image size is %s" % (imgsize))
print ("%d classes" % (nclass))

['trainlabel', 'imgsize', 'trainimg', 'testimg', 'testlabel', 'use_gray']
408 train images loaded
273 test images loaded
4096 dimensional input
Image size is [64 64]
4 classes


# DEFINE NETWORK

In [3]:
tf.set_random_seed(0)
# Parameters
learning_rate   = 0.001
training_epochs = 200
batch_size      = ntrain
display_step    = 20

# Network Parameters
n_hidden_1 = 128 # 1st layer num features
n_hidden_2 = 128 # 2nd layer num features
n_input    = dim # data input 
n_classes  = nclass # total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2']))
    return tf.matmul(layer_2, _weights['out']) + _biases['out']
    
# Store layers weight & bias
stddev = 0.1 # <== This greatly affects accuracy!! 
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
print ("Network Ready to Go!")

Network Ready to Go!


# DEFINE FUNCTIONS

In [4]:
# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) 
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, "float"))

# Initializing the variables
init = tf.initialize_all_variables()
print ("Functions ready")

Functions ready


# OPTIMIZE

In [5]:
# Launch the graph
sess = tf.Session()
sess.run(init)

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(ntrain/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        randidx = np.random.randint(ntrain, size=batch_size)
        batch_xs = trainimg[randidx, :]
        batch_ys = trainlabel[randidx, :]   
        # Fit training using batch data
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
        # Compute average loss
        avg_cost += sess.run(cost, 
                feed_dict={x: batch_xs, y: batch_ys})/total_batch
        # Display logs per epoch step
    if epoch % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % 
               (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys})
        print (" Training accuracy: %.3f" % (train_acc))
        test_acc = sess.run(accr, feed_dict={x: testimg, y: testlabel})
        print (" Test accuracy: %.3f" % (test_acc))

print ("Optimization Finished!")

Epoch: 000/200 cost: 1.271932960
 Training accuracy: 0.723
 Test accuracy: 0.707
Epoch: 020/200 cost: 0.485024929
 Training accuracy: 0.831
 Test accuracy: 0.817
Epoch: 040/200 cost: 0.269245446
 Training accuracy: 0.926
 Test accuracy: 0.857
Epoch: 060/200 cost: 0.158787951
 Training accuracy: 0.973
 Test accuracy: 0.883
Epoch: 080/200 cost: 0.054250188
 Training accuracy: 0.993
 Test accuracy: 0.905
Epoch: 100/200 cost: 0.034250565
 Training accuracy: 0.998
 Test accuracy: 0.908
Epoch: 120/200 cost: 0.016527731
 Training accuracy: 1.000
 Test accuracy: 0.905
Epoch: 140/200 cost: 0.013574737
 Training accuracy: 1.000
 Test accuracy: 0.894
Epoch: 160/200 cost: 0.006475344
 Training accuracy: 1.000
 Test accuracy: 0.894
Epoch: 180/200 cost: 0.004590446
 Training accuracy: 1.000
 Test accuracy: 0.894
Optimization Finished!


# CLOSE SESSION

In [6]:
sess.close()
print ("Session closed.")

Session closed.
